# Install Packages

In [ ]:
!pip install deepchem torch-geometric transformers lightning

In [3]:
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html

Looking in links: https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.1/355.1 MB 2.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.6/443.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0


In [42]:
import deepchem as dc
import torch

# QM9 DeepChem Dataset Info
QM9_TASKS = [
    "mu", "alpha", "homo", "lumo", "gap", "r2", "zpve", "cv", "u0", "u298",
    "h298", "g298"
]

# MolGraphConvFeaturizer Models

In [112]:
featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)

tasks, datasets, transformers = dc.molnet.load_qm9(featurizer=featurizer)

train, val, test = datasets

In [104]:
train.w.shape

(105576, 12)

In [32]:
train.X[6] # 30, 11

GraphData(node_features=[9, 30], edge_index=[2, 18], edge_features=[18, 11])

In [106]:
train

<DiskDataset X.shape: (105576,), y.shape: (105576, 12), w.shape: (105576, 12), task_names: ['mu' 'alpha' 'homo' ... 'u298' 'h298' 'g298']>

In [109]:
def filter_dataset(dataset, selected_indicies):
    X = dataset.X
    y = dataset.y[:, selected_indicies]
    w = dataset.w[:, selected_indicies]
    ids = dataset.ids
    return dc.data.NumpyDataset(X, y, w, ids)

In [138]:
# Choose only tasks= ["homo", "lumo", "gap"]
from deepchem.trans import NormalizationTransformer



selected_tasks = ["homo", "lumo", "gap"]
selected_indicies = [tasks.index(task) for task in selected_tasks]

train_filtered = filter_dataset(train, selected_indicies)
val_filtered = filter_dataset(val, selected_indicies)
test_filtered = filter_dataset(test, selected_indicies)

transformers = [NormalizationTransformer(
    transform_y=True,
    dataset=train_filtered,
    move_mean=True
)]

In [122]:
train_filtered.X[0]

GraphData(node_features=[16, 30], edge_index=[2, 32], edge_features=[32, 11])

In [123]:
len(selected_tasks)

3

## PagtnModel

In [20]:
!pip install dgllife

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 1.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 8.2 MB/s eta 0:00:00


In [137]:
from deepchem.models.optimizers import Adam
# tasks = [ "mu", "alpha", "homo", "lumo", "gap", "r2", "zpve", "cv", "u0", "u298", "h298", "g298" ]  # Multiple tasks

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

optimizer = Adam(learning_rate=0.001)

model = dc.models.PagtnModel(
    n_tasks=len(selected_tasks),
    number_atom_features=30, # Should match the dimension of featurized dataset
    number_bond_features=11, # ''
    mode='regression',
    batch_size=16384,
    # learning_rate=0.025,
    optimizer = optimizer,
    hidden_features=64,  # Hidden dimension size
    output_node_features=64,  # Output features before final layer
    num_layers=3,        # Number of PAGTN layers
    num_heads=4,         # Number of attention heads
    dropout=0.2,
    max_path_length=5,   # Should match featurizer's max_length
    path_hidden_dim=64, # Hidden dimension for path embeddings
    device=device,
    model_dir="./models/PagtnModel/"
)

Using device: cuda:0


In [95]:
metrics = [
    dc.metrics.Metric(dc.metrics.mae_score),
    dc.metrics.Metric(dc.metrics.pearson_r2_score),
    dc.metrics.Metric(dc.metrics.mean_squared_error)
]

### Lightning (doesnt work)

In [67]:
from deepchem.models.lightning import DCLightningModule, DCLightningDatasetModule
import pytorch_lightning as L

lightning_model = DCLightningModule(model)
train_data_module = DCLightningDatasetModule(train, model.batch_size)
valid_data_module = DCLightningDatasetModule(val, model.batch_size)

trainer = L.Trainer(
    accelerator='gpu',  # Use GPU
    devices=1,  # Use 1 GPU (change to more if you have multiple)
    max_epochs=50,
    # precision=16,  # Use mixed precision for faster training
    callbacks=[
        L.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            mode='min',
            verbose=True
        ),
        L.callbacks.ModelCheckpoint(
            monitor='val_loss',
            dirpath='lightning_checkpoints/',
            filename='pagtn-{epoch:02d}-{val_loss:.2f}',
            save_top_k=3,
            mode='min'
        ),
        L.callbacks.LearningRateMonitor(logging_interval='epoch')
    ],
    # logger=L.loggers.TensorBoardLogger('lightning_logs/', name='pagtn_qm9'),
    logger=L.loggers.CSVLogger('lightning_logs/', name='pagtn_qm9'),
    log_every_n_steps=10,
    gradient_clip_val=0.5,  # Gradient clipping for stability
    enable_progress_bar=True,
    deterministic=True  # For reproducibility
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
print("Starting Lightning training on GPU...")
trainer.fit(lightning_model, train_data_module, valid_data_module)

### Regular

In [72]:
torch.use_deterministic_algorithms(False)

In [ ]:
best_valid_mae = float('inf')
patience = 20
current_patience = 0
training_history = []

torch.cuda.empty_cache()

print("Base MAE")
valid_scores = model.evaluate(val_filtered, metrics, transformers)
valid_mae = valid_scores['mae_score']
print(f"Valid_mae: {valid_mae}")

print("Training PagtnModel...")
for epoch in range(200):
    print(f"Epoch #{epoch + 1}", end='\t')
    if torch.cuda.is_available():
        print(f"\nGPU Memory allocated: {torch.cuda.memory_allocated(0)/1e9:.2f} GB")
    
    loss = model.fit(train_filtered, nb_epoch=1, deterministic=False)
    training_history.append(loss)
    
    # Evaluate on validation set
    valid_scores = model.evaluate(val_filtered, metrics, transformers)
    valid_mae = valid_scores['mae_score']

    print(f"Valid_mae: {valid_mae}")
    
    if epoch % 10 == 0:
        train_scores = model.evaluate(train_filtered, metrics, transformers)
        print(f"Epoch {epoch}:")
        print(f"  Train MAE: {train_scores['mae_score']:.4f}, "
              f"R2: {train_scores['pearson_r2_score']:.4f}")
        print(f"  Valid MAE: {valid_mae:.4f}, "
              f"R2: {valid_scores['pearson_r2_score']:.4f}")
    
    # Early stopping
    if valid_mae < best_valid_mae:
        best_valid_mae = valid_mae
        current_patience = 0
        # Save best model
        # model.save()
        torch.save(model.model.state_dict(), "best_model_checkpoint.pth")
    else:
        current_patience += 1
        print(f"MAE did not decrease, current patience: {current_patience}")
        if current_patience >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

# Load best model for evaluation
# model.restore()

Base MAE
Valid_mae: 3.482545019443993
Training PagtnModel...
Epoch #1	
GPU Memory allocated: 0.02 GB
Valid_mae: 1.3255443662811253
Epoch 0:
  Train MAE: 1.3236, R2: 0.0001
  Valid MAE: 1.3255, R2: 0.0001
Epoch #2	
GPU Memory allocated: 0.02 GB


In [ ]:
model.model.state_dict()

In [93]:
torch.cuda.empty_cache()